<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/europeanoption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import QuantLib as ql
import unittest
import math

# --- Helper Utilities (assuming they are defined as in previous examples) ---
def flat_rate_py(evaluation_date_or_quote, forward_rate_or_dc, day_counter_or_none=None):
    if isinstance(evaluation_date_or_quote, ql.Date):
        evaluation_date = evaluation_date_or_quote
        forward_rate_obj = forward_rate_or_dc
        day_counter = day_counter_or_none
    else:
        evaluation_date = ql.Settings.instance().evaluationDate
        forward_rate_obj = evaluation_date_or_quote
        day_counter = forward_rate_or_dc
    if isinstance(forward_rate_obj, ql.Quote):
        quote_handle = ql.QuoteHandle(forward_rate_obj)
    elif isinstance(forward_rate_obj, float):
        quote_handle = ql.QuoteHandle(ql.SimpleQuote(forward_rate_obj))
    else:
        quote_handle = forward_rate_obj
    return ql.FlatForward(evaluation_date, quote_handle, day_counter)

def flat_vol_py(evaluation_date_or_quote, vol_level_or_dc, day_counter_or_none=None):
    if isinstance(evaluation_date_or_quote, ql.Date):
        evaluation_date = evaluation_date_or_quote
        vol_level_obj = vol_level_or_dc
        day_counter = day_counter_or_none
    else:
        evaluation_date = ql.Settings.instance().evaluationDate
        vol_level_obj = evaluation_date_or_quote
        day_counter = vol_level_or_dc
    if isinstance(vol_level_obj, ql.Quote):
        vol_quote_handle = ql.QuoteHandle(vol_level_obj)
    elif isinstance(vol_level_obj, float):
        vol_quote_handle = ql.QuoteHandle(ql.SimpleQuote(vol_level_obj))
    else:
        vol_quote_handle = vol_level_obj
    return ql.BlackConstantVol(evaluation_date, ql.NullCalendar(), vol_quote_handle, day_counter)

def time_to_days_py(t, basis=360): # C++ default in this test for timeToDays is 360
    return int(t * basis + 0.5)

def exercise_type_to_string_py(exercise):
    if isinstance(exercise, ql.EuropeanExercise): return "European"
    return "UnknownExercise"

def payoff_type_to_string_py(payoff):
    if isinstance(payoff, ql.PlainVanillaPayoff): return "PlainVanilla"
    if isinstance(payoff, ql.CashOrNothingPayoff): return "CashOrNothing"
    if isinstance(payoff, ql.AssetOrNothingPayoff): return "AssetOrNothing"
    if isinstance(payoff, ql.GapPayoff): return "Gap"
    return "UnknownPayoff"

def relative_error_py(x1, x2, reference):
    if abs(reference) > 1.0e-12: # Avoid division by zero or near-zero reference
        return abs(x1 - x2) / abs(reference)
    else: # Fallback for zero or very small reference
        return abs(x1 - x2)


class EuropeanOptionData:
    def __init__(self, type_opt, strike, s, q, r, t, v, result, tol=0.0): # tol often not in Haug data
        self.type = type_opt; self.strike = strike; self.s = s; self.q = q; self.r = r
        self.t = t; self.v = v; self.result = result; self.tol = tol if tol != 0.0 else 1.0e-4 # Default tol

# --- EngineType Enum Python equivalent ---
ENGINE_ANALYTIC = "Analytic"
ENGINE_JR = "JR"
ENGINE_CRR = "CRR"
ENGINE_EQP = "EQP"
ENGINE_TGEO = "TGEO"
ENGINE_TIAN = "TIAN"
ENGINE_LR = "LR"
ENGINE_JOSHI = "JOSHI"
ENGINE_FD = "FiniteDifferences"
ENGINE_INTEGRAL = "Integral"
ENGINE_MC_PSEUDO = "PseudoMonteCarlo"
ENGINE_MC_QUASI = "QuasiMonteCarlo"
ENGINE_FFT = "FFT"


class EuropeanOptionTests(unittest.TestCase):

    def setUp(self):
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        self.today = ql.Date(15, ql.May, 2007) # Arbitrary fixed date for consistency
        ql.Settings.instance().evaluationDate = self.today

        self.dc = ql.Actual360()
        # Market data quotes and handles are initialized here for clarity
        # In C++ they are often global or part of a struct initialized once
        self.spot_q = ql.SimpleQuote(0.0)
        self.q_rate_q = ql.SimpleQuote(0.0)
        self.r_rate_q = ql.SimpleQuote(0.0)
        self.vol_q = ql.SimpleQuote(0.0)

        self.spot_h = ql.QuoteHandle(self.spot_q)
        self.qTS_h = ql.YieldTermStructureHandle(flat_rate_py(self.today, self.q_rate_q, self.dc))
        self.rTS_h = ql.YieldTermStructureHandle(flat_rate_py(self.today, self.r_rate_q, self.dc))
        self.volTS_h = ql.BlackVolTermStructureHandle(flat_vol_py(self.today, self.vol_q, self.dc))


    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.saved_eval_date

    def _make_process(self, u_q_h, q_ts_h, r_ts_h, vol_ts_h):
        return ql.BlackScholesMertonProcess(u_q_h, q_ts_h, r_ts_h, vol_ts_h)

    def _make_option(self, payoff, exercise, u_q_h, q_ts_h, r_ts_h, vol_ts_h,
                     engine_type_str, binomial_steps=None, mc_samples=None):

        process = self._make_process(u_q_h, q_ts_h, r_ts_h, vol_ts_h)
        engine = None

        if engine_type_str == ENGINE_ANALYTIC:
            engine = ql.AnalyticEuropeanEngine(process)
        elif engine_type_str == ENGINE_JR:
            engine = ql.BinomialVanillaEngine(process, "JarrowRudd", binomial_steps)
        elif engine_type_str == ENGINE_CRR:
            engine = ql.BinomialVanillaEngine(process, "CoxRossRubinstein", binomial_steps)
        elif engine_type_str == ENGINE_EQP:
            engine = ql.BinomialVanillaEngine(process, "AdditiveEQPBinomialTree", binomial_steps)
        elif engine_type_str == ENGINE_TGEO:
            engine = ql.BinomialVanillaEngine(process, "Trigeorgis", binomial_steps)
        elif engine_type_str == ENGINE_TIAN:
            engine = ql.BinomialVanillaEngine(process, "Tian", binomial_steps)
        elif engine_type_str == ENGINE_LR:
            engine = ql.BinomialVanillaEngine(process, "LeisenReimer", binomial_steps)
        elif engine_type_str == ENGINE_JOSHI:
            engine = ql.BinomialVanillaEngine(process, "Joshi4", binomial_steps)
        elif engine_type_str == ENGINE_FD:
            # C++ uses binomialSteps for timeSteps and samples for gridPoints
            engine = ql.FdBlackScholesVanillaEngine(process, binomial_steps, mc_samples)
        elif engine_type_str == ENGINE_INTEGRAL:
            engine = ql.IntegralEngine(process)
        elif engine_type_str == ENGINE_MC_PSEUDO:
            engine = ql.MCEuropeanEngine(process, "PseudoRandom", timeSteps=1,
                                         requiredSamples=mc_samples, seed=42)
        elif engine_type_str == ENGINE_MC_QUASI:
            engine = ql.MCEuropeanEngine(process, "LowDiscrepancy", timeSteps=1,
                                         requiredSamples=mc_samples)
        elif engine_type_str == ENGINE_FFT:
            engine = ql.FFTVanillaEngine(process)
        else:
            raise ValueError(f"Unknown engine type: {engine_type_str}")

        option = ql.EuropeanOption(payoff, exercise) # EuropeanOption explicitly
        option.setPricingEngine(engine)
        return option

    def _update_market_data(self, s, q, r, v):
        # Updates the quotes linked to the handles
        self.spot_q.setValue(s)
        self.q_rate_q.setValue(q)
        self.r_rate_q.setValue(r)
        self.vol_q.setValue(v)
        # Term structures and process will pick up new quote values automatically

    def _report_failure(self, greek_name, payoff, exercise, s, q, r, today,
                        v, expected, calculated, error, tolerance):
        # Ensure rates and vols are floats for formatting
        q_float = q.value() if isinstance(q, ql.Quote) else q
        r_float = r.value() if isinstance(r, ql.Quote) else r
        v_float = v.value() if isinstance(v, ql.Quote) else v
        msg = (f"{exercise_type_to_string_py(exercise)} "
               f"{payoff.optionType()} option with {payoff_type_to_string_py(payoff)} payoff:\n"
               f"    spot value:       {s}\n    strike:           {payoff.strike()}\n"
               f"    dividend yield:   {q_float:.6f}\n    risk-free rate:   {r_float:.6f}\n"
               f"    reference date:   {today}\n    maturity:         {exercise.lastDate()}\n"
               f"    volatility:       {v_float:.6f}\n\n"
               f"    expected   {greek_name}: {expected}\n    calculated {greek_name}: {calculated}\n"
               f"    error:            {error}\n    tolerance:        {tolerance}")
        self.fail(msg)


    def test_values_haug(self): # Renamed from testValues to be specific
        print("Testing European option values against Haug's data...")
        haug_values = [
            EuropeanOptionData(ql.Option.Call, 65.00, 60.00, 0.00, 0.08, 0.25, 0.30, 2.1334, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  95.00, 100.00,0.05, 0.10, 0.50, 0.20, 2.4648, 1.0e-4),
            # ... (add more Haug values if needed, using a subset for brevity)
            EuropeanOptionData(ql.Option.Call, 40.00, 42.00, 0.08, 0.04, 0.75, 0.35, 5.0975, 1.0e-4)
        ]

        # Use a fixed eval date for this test
        current_eval_date = ql.Date(10, ql.January, 2020) # Example
        ql.Settings.instance().evaluationDate = current_eval_date

        # Update handles with new eval date
        self.qTS_h.linkTo(flat_rate_py(current_eval_date, self.q_rate_q, self.dc))
        self.rTS_h.linkTo(flat_rate_py(current_eval_date, self.r_rate_q, self.dc))
        self.volTS_h.linkTo(flat_vol_py(current_eval_date, self.vol_q, self.dc))
        # Recreate process with new eval date context in TS
        process = self._make_process(self.spot_h, self.qTS_h, self.rTS_h, self.volTS_h)


        for val_data in haug_values:
            self._update_market_data(val_data.s, val_data.q, val_data.r, val_data.v)

            payoff = ql.PlainVanillaPayoff(val_data.type, val_data.strike)
            ex_date = current_eval_date + time_to_days_py(val_data.t) # Assuming Act/360 for t conversion
            exercise = ql.EuropeanExercise(ex_date)

            engine_analytic = ql.AnalyticEuropeanEngine(process)
            option_analytic = ql.EuropeanOption(payoff, exercise)
            option_analytic.setPricingEngine(engine_analytic)

            calculated_analytic = option_analytic.NPV()
            error_analytic = abs(calculated_analytic - val_data.result)
            if error_analytic > val_data.tol:
                self._report_failure("value (Analytic)", payoff, exercise, val_data.s, val_data.q, val_data.r,
                                     current_eval_date, val_data.v, val_data.result, calculated_analytic,
                                     error_analytic, val_data.tol)

            # Test with FD engine as well (C++ test does this in same loop)
            engine_fd = ql.FdBlackScholesVanillaEngine(process, 200, 400) # timeSteps, gridPoints
            option_fd = ql.EuropeanOption(payoff, exercise)
            option_fd.setPricingEngine(engine_fd)
            calculated_fd = option_fd.NPV()
            error_fd = abs(calculated_fd - val_data.result)
            tol_fd = 1.0e-3 # C++ uses larger tolerance for FD
            if error_fd > tol_fd:
                self._report_failure("value (FD)", payoff, exercise, val_data.s, val_data.q, val_data.r,
                                     current_eval_date, val_data.v, val_data.result, calculated_fd,
                                     error_fd, tol_fd)

    # ... testGreekValues (Haug's greeks) would be similar, testing option.delta(), .gamma() etc.

    def _test_engine_consistency_runner(self, engine_type_str, binomial_steps, mc_samples,
                                       tolerance_map, test_greeks=False):
        print(f"Testing engine consistency for {engine_type_str}...")
        # Test options
        types = [ql.Option.Call, ql.Option.Put]
        strikes = [75.0, 100.0, 125.0]
        lengths = [1] # Years
        # Test data
        underlyings = [100.0]
        q_rates = [0.00, 0.05]
        r_rates = [0.01, 0.05, 0.15]
        vols = [0.11, 0.50, 1.20]

        # Use a fixed eval date for this test run
        current_eval_date = ql.Date(10, ql.January, 2020)
        ql.Settings.instance().evaluationDate = current_eval_date
        self.qTS_h.linkTo(flat_rate_py(current_eval_date, self.q_rate_q, self.dc))
        self.rTS_h.linkTo(flat_rate_py(current_eval_date, self.r_rate_q, self.dc))
        self.volTS_h.linkTo(flat_vol_py(current_eval_date, self.vol_q, self.dc))

        for opt_type in types:
            for strike in strikes:
                for length in lengths:
                    ex_date = current_eval_date + ql.Period(length * 360, ql.Days) # Approx. year
                    exercise = ql.EuropeanExercise(ex_date)
                    payoff = ql.PlainVanillaPayoff(opt_type, strike)

                    ref_option = self._make_option(payoff, exercise, self.spot_h, self.qTS_h, self.rTS_h, self.volTS_h,
                                                   ENGINE_ANALYTIC)
                    option_to_test = self._make_option(payoff, exercise, self.spot_h, self.qTS_h, self.rTS_h, self.volTS_h,
                                                       engine_type_str, binomial_steps, mc_samples)

                    for u_val in underlyings:
                        for q_val in q_rates:
                            for r_val in r_rates:
                                for v_val in vols:
                                    self._update_market_data(u_val, q_val, r_val, v_val)

                                    expected = {"value": ref_option.NPV()}
                                    calculated = {"value": option_to_test.NPV()}

                                    if test_greeks and option_to_test.NPV() > self.spot_q.value() * 1.0e-5:
                                        expected["delta"] = ref_option.delta()
                                        expected["gamma"] = ref_option.gamma()
                                        expected["theta"] = ref_option.theta()
                                        calculated["delta"] = option_to_test.delta()
                                        calculated["gamma"] = option_to_test.gamma()
                                        calculated["theta"] = option_to_test.theta()

                                    for greek_name, calc_val in calculated.items():
                                        exp_val = expected[greek_name]
                                        tol = tolerance_map[greek_name]
                                        error = relative_error_py(exp_val, calc_val, u_val)
                                        if error > tol:
                                            self._report_failure(f"{greek_name} ({engine_type_str})", payoff, exercise,
                                                                 u_val, q_val, r_val, current_eval_date, v_val,
                                                                 exp_val, calc_val, error, tol)

    def test_jr_binomial_engines(self):
        # @unittest.skipIf(True, "Potentially slow test (JR Binomial)")
        tol_map = {"value": 0.002, "delta": 1.0e-3, "gamma": 1.0e-4, "theta": 0.03}
        self._test_engine_consistency_runner(ENGINE_JR, 251, None, tol_map, True)

    def test_crr_binomial_engines(self):
        tol_map = {"value": 0.02, "delta": 1.0e-3, "gamma": 1.0e-4, "theta": 0.03}
        self._test_engine_consistency_runner(ENGINE_CRR, 501, None, tol_map, True)

    # ... Similar test methods for EQP, TGEO, TIAN, LR, JOSHI, FD, Integral, MC, QMC, FFT ...
    # Each would call _test_engine_consistency_runner with appropriate engine_type_str and tolerances.

    def test_implied_vol(self):
        print("Testing European option implied volatility...")
        # Setup (subset for brevity)
        max_evaluations = 100; tolerance = 1.0e-6
        types = [ql.Option.Call, ql.Option.Put]
        strikes = [99.5, 100.0, 100.5]
        lengths_days = [360] # Days for exercise date
        underlyings = [100.0]
        q_rates = [0.05]; r_rates = [0.05]; vols = [0.20]

        current_eval_date = ql.Date(10, ql.January, 2020) # Fixed eval date
        ql.Settings.instance().evaluationDate = current_eval_date
        self.qTS_h.linkTo(flat_rate_py(current_eval_date, self.q_rate_q, self.dc))
        self.rTS_h.linkTo(flat_rate_py(current_eval_date, self.r_rate_q, self.dc))
        self.volTS_h.linkTo(flat_vol_py(current_eval_date, self.vol_q, self.dc))
        process = self._make_process(self.spot_h, self.qTS_h, self.rTS_h, self.volTS_h)


        for opt_type in types:
            for strike in strikes:
                for length_d in lengths_days:
                    ex_date = current_eval_date + ql.Period(length_d, ql.Days)
                    exercise = ql.EuropeanExercise(ex_date)
                    payoff = ql.PlainVanillaPayoff(opt_type, strike)

                    option = self._make_option(payoff, exercise, self.spot_h, self.qTS_h, self.rTS_h, self.volTS_h, ENGINE_ANALYTIC)

                    for u_val in underlyings:
                        for q_val in q_rates:
                            for r_val in r_rates:
                                for v_val in vols:
                                    self._update_market_data(u_val, q_val, r_val, v_val)

                                    option_value = option.NPV()
                                    if abs(option_value) < 1e-12 : continue # Skip if value is zero

                                    # Shift guess for implied vol
                                    self.vol_q.setValue(v_val * 0.5) # Update vol quote for process
                                    if abs(option_value - option.NPV()) <= 1.0e-12: # Flat price vs vol
                                        self.vol_q.setValue(v_val) # Reset vol
                                        continue
                                    self.vol_q.setValue(v_val) # Reset vol for process used in impliedVol calculation

                                    impl_vol_val = 0.0
                                    try:
                                        impl_vol_val = option.impliedVolatility(option_value, process, tolerance, max_evaluations)
                                    except RuntimeError as e:
                                        self.fail(f"Implied vol calculation failed: {e} for OptVal={option_value}")

                                    self.assertAlmostEqual(impl_vol_val, v_val, delta=tolerance,
                                                           msg=f"Implied vol mismatch for K={strike}, T={length_d}d: "
                                                               f"Got {impl_vol_val}, Exp {v_val}")
                                    # Second check if difference is not too small
                                    if abs(impl_vol_val - v_val) > tolerance:
                                        self.vol_q.setValue(impl_vol_val)
                                        value2 = option.NPV()
                                        error_val_check = relative_error_py(option_value, value2, u_val)
                                        self.assertLessEqual(error_val_check, tolerance,
                                                             msg="Value mismatch after re-pricing with implied vol")

    # ... testImpliedVolWithDividends, testImpliedVolContainment, testLocalVolatility, ...
    # ... testAnalyticEngineDiscountCurve, testPDESchemes, testFdEngineWithNonConstantParameters, ...
    # ... testDouglasVsCrankNicolson ...
    # These would be extensive but follow similar patterns of setting up specific scenarios
    # and comparing results.

if __name__ == '__main__':
    print("Testing QuantLib " + ql.__version__)
    unittest.main(argv=['first-arg-is-ignored'], exit=False)